## NIMOハンズオン (関数の最適化)

Googleドライブを読み込めるように設定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


モジュールのパスを設定

In [9]:
import sys
sys.path.append('/content/drive/MyDrive/NIMO_handson')

NIMOをインストール

In [10]:
!pip install nimo

In [11]:
import nimo

### AIとロボットのサイクルの回数を設定

In [12]:
CyclesNum = 51

### 目的関数の次元を設定

In [13]:
ObjectivesNum = 1

### 実験条件候補データファイル名および提案条件のファイル名を設定

In [14]:
candidates_file = "/content/drive/MyDrive/NIMO_handson/candidates_SO_Sphere.csv"

proposals_file = "/content/drive/MyDrive/NIMO_handson/proposals.csv"

### 結果を格納する場所を作成

In [15]:
res_history = nimo.history(input_file = candidates_file,
                             num_objectives = ObjectivesNum)

### 最適化計算

In [18]:
for K in range(CyclesNum):

    #最初のサイクルではランダムに実験条件を提案させる
    if K == 0:
        nimo.selection(method = "RE",
                       input_file = candidates_file,
                       output_file = proposals_file,
                       num_objectives = ObjectivesNum,
                       num_proposals = 5,
                       re_seed = 111)

    #2回目以降のサイクルではAIを使用して実験条件を提案
    #PHYSBO: ベイズ最適化
    #BLOX: 無目的探索
    #PDC: 相図探索
    #RE: ランダム
    else:
        nimo.selection(method = "PHYSBO",
                       input_file = candidates_file,
                       output_file = proposals_file,
                       num_objectives = ObjectivesNum,
                       num_proposals = 1)

    #実験条件に従い，ロボット実験のインプットファイルを作成する
    #今回は，inputfile.txtファイルを作成するだけのモジュール
    import preparation_input_functions
    preparation_input_functions.ORIGINAL(input_file = proposals_file,
                                         input_folder = "/content/drive/MyDrive/NIMO_handson/").perform()


    #実験終了後，測定データを解析し，目的関数をアップデートする
    #今回は，関数値を計算し，目的関数とするanalysis_output_functions.pyを利用する
    import analysis_output_functions_SO_Sphere
    analysis_output_functions_SO_Sphere.ORIGINAL(input_file = proposals_file,
                                       output_file = candidates_file,
                                       num_objectives = ObjectivesNum,
                                       output_folder = "/content/drive/MyDrive/NIMO_handson/").perform()

    #結果をプロットするためのhistoryを格納する
    res_history = nimo.history(input_file = candidates_file,
                               num_objectives = ObjectivesNum,
                               itt = K,
                               history_file = res_history)

    #結果のヒストグラムを作成するモジュールを読み出す
    if K % 5 == 0:
        import time
        time.sleep(1)
        nimo.visualization.plot_distribution.plot(input_file = candidates_file,
                                                  num_objectives = ObjectivesNum,
                                                  fig_folder = "/content/drive/MyDrive/NIMO_handson/fig")

    #結果の相図を作成するモジュールを読み出す
    #if K % 5 == 0:
    #    import time
    #    time.sleep(1)
    #    nimo.visualization.plot_phase_diagram.plot(input_file = candidates_file,
    #                                               fig_folder = "/content/drive/MyDrive/NIMO_handson/fig")


Start selection of proposals by RE!
Proposals
###
number = 1
actions =  871
proposal =  [-1.32 -1.84]
###
###
number = 2
actions =  1294
proposal =  [-1.   -1.24]
###
###
number = 3
actions =  2023
proposal =  [-0.44 -0.64]
###
###
number = 4
actions =  795
proposal =  [-1.4 -0.8]
###
###
number = 5
actions =  1628
proposal =  [-0.76 -0.12]
###
Finish selection of proposals!
Start preparation input!
Finish preparation input!
Start analysis output!
Finish analysis output!
Start selection of proposals by PHYSBO!
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood 8.294134747090624
50 -th epoch marginal likelihood 8.212845829226014
100 -th epoch marginal likelihood 8.147557899113004
150 -th epoch marginal likelihood 8.095848195692358
200 -th epoch marginal likelihood 8.055382253448283
250 -th epoch marginal likelihood 8.023535531145185
300 -th epoch marginal likelihood 7.997865167321418
350 -th epoch marginal likelih

### 目的関数の推移を出力

In [19]:
nimo.visualization.plot_history.cycle(input_file = res_history,
                                      num_cycles = CyclesNum,
                                      fig_folder = "/content/drive/MyDrive/NIMO_handson/fig/")